In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv('train_set_100_2.csv',encoding='utf-8')
labels = pd.factorize(data.sucs.unique())[0]
data.head()

,ictihats,sucs
0,hük hükümlülük müsader tasfi yerel mahkemeç ve...,5607 sayılı kanuna muhalefet
1,taraf görüle dava ankar fikri hak hukuk mahkem...,undefined
2,taraf muhakemes bölge adli mahkemes hukuk dair...,undefined
3,5271 139 140 5237 53 192 any mah 08102015 2014...,undefined
4,mahke bölge adli mahkemes ceza daires hüküm ce...,uyuşturucu madde ticareti yapma


In [3]:
X =(data.ictihats.astype(str))
print(type(X))
Y = (pd.Series(labels))
X = X.values.reshape(2785,10)
# X = X.values.reshape(X.values.shape[0:2])
X = X.transpose()
# le = LabelEncoder()
# Y = le.fit_transform(Y)
Y = Y.values.reshape(-1)
print(X.shape)
print(Y.shape)

<class 'pandas.core.series.Series'>
(10, 2785)
(10,)


In [4]:
# X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)
X_train = pd.Series(X[:int(len(data)*0.8)].ravel().squeeze())
X_test = pd.Series(X[int(len(data)*0.8):].ravel().squeeze())
Y_train = pd.Series(Y[:int(len(data)*0.8)].ravel().squeeze())
Y_test = pd.Series(Y[:int(len(data)*0.8)].ravel().squeeze())

In [5]:
max_words = 150000
max_len = 500
tok = Tokenizer(num_words=max_words)
#print(len(X_train))
#x_train = pd.Series(X_train.ravel().squeeze()) 
#print((len(x_train)))
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = keras.utils.pad_sequences(sequences,maxlen=max_len)
print((len(sequences_matrix)))

27850


In [6]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,300,input_length=max_len)(inputs)
    layer = LSTM(128)(layer)
    layer = Dense(10,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(10,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [7]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 500)]             0         
                                                                 
 embedding (Embedding)       (None, 500, 300)          45000000  
                                                                 
 lstm (LSTM)                 (None, 128)               219648    
                                                                 
 FC1 (Dense)                 (None, 10)                1290      
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 out_layer (Dense)           (None, 10)                110   

In [8]:
print((len(sequences_matrix)))
model.fit(sequences_matrix,Y_train,batch_size=64,epochs=20,
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

27850


ValueError: Data cardinality is ambiguous:
  x sizes: 27850
  y sizes: 10
Make sure all arrays contain the same number of samples.